Self link: https://colab.research.google.com/drive/1BNok28YCY5ar8PCHUBGIrVzS1QS3vTDo?usp=sharing

In [ ]:
import torch
import torch.onnx
import onnxruntime
import numpy as np

In [ ]:
onnx_path = 'onnx_model.onnx'
tf_path = 'tf_saved_model'
tflite_path = 'tflite_model.tflite' # name of the output tflite model

# 1. Load pytorch model

Upload pytorch model in colab

In [ ]:
# Load saved pytorch model
model = torch.load('pytorch_model.pt', map_location=torch.device('cpu'))

# Input to the model with shape (1,3,300,300)
x = torch.randn(1,3,300,300, requires_grad=True)

# Output of the model used to validate results with onnx model
torch_out = model(x)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set

# 2. Export pytorch model to onnx

Executes the model, records a trace of what operators are used to compute the outputs. The input tensor ('x') is required to run the model, x can be random and must have the correct type and size

In [ ]:
torch.onnx.export(model, 
                  x,
                  onnx_path,
                  export_params=True,
                  input_names=["input"], 
                  output_names=["output"])

## Check the ONNX model with ONNX's API 

Verify the model's structure and confirm that the model has a valid schema


In [ ]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

# Print a human readable representation of the graph
# print(onnx.helper.printable_graph(model.graph))

## Test results of exported model with ONNXRuntime

In [ ]:
ort_session = onnxruntime.InferenceSession(onnx_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results with the given precision
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [ ]:
import onnx

from onnx_tf.backend import prepare

import tensorflow as tf

# 3. Export ONNX model to TF

In [ ]:
onnx_model = onnx.load(onnx_path)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(tf_path)  # export the model

# 4. Export TF model to TFLite

In [ ]:
# Convert the tf model to tflite
converter = tf.lite.TFLiteConverter.from_saved_model(tf_path) # path to the SavedModel directory
converter.optimizations = {tf.lite.Optimize.DEFAULT} # if size exceeds 40MB, try optimizing
tflite_model = converter.convert()

# Save the tflite model.
with open(tflite_path, 'wb') as f:
  f.write(tflite_model)